Integrantes:

Castillo Ilabaca, Sebastian
Corti, Gastón
Marcos, Videla

4 - Implemente algún método de búsqueda. Puedes elegir cualquiera menos búsqueda en anchura primero (el desarrollado en clase). Sos libre de elegir cualquiera de los vistos en clases, o inclusive buscar nuevos -- Celdas 1 a 6 inclusive

6 - A nivel implementación, ¿qué tiempo y memoria ocupa el algoritmo? (Se recomienda correr 10 veces y calcular promedio y desvío estándar de las métricas). -- Celdas 7 y 8

7 - Si la solución óptima es 2k -1 movimientos con k igual al número de discos. Qué tan lejos está la solución del algoritmo implementado de esta solución óptima (se recomienda correr al menos 10 veces y usar el promedio de trayecto usado) -- Celda 9

Nota adicional : El código de la celda 10 genera los archivos json para la simulación del experimento.

ENUNCIADO 4 - IMPLEMENTACIÓN DE METODO DE BUSQUEDA EN PROFUNDIDAD (COLA LIFO)

In [1]:
import sys
# agregando la ruta al modulo
sys.path.append(r'C:\Users\sebas\Hanoi_Tower_TP1\intro_ia\clase2\hanoi_tower')

# importando los componentes deseados de los modulos
from aima_libs.hanoi_states import StatesHanoi
from aima_libs.hanoi_states import ActionHanoi
from aima_libs.hanoi_states import ProblemHanoi
from aima_libs.tree_hanoi import NodeHanoi

In [2]:
# 4 - Implemente algún método de búsqueda. Puedes elegir cualquiera menos búsqueda en anchura primero (el desarrollado en clase). 

def depth_first_search(number_disks=5):
    # Definiendo el problema
    list_disks = [i for i in range(number_disks, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    # Definiendo la raiz
    root = NodeHanoi(state=initial_state)
    
    # Definiendo la estructura de cola LIFO
    frontier = [NodeHanoi(problem.initial)]

    # Definiendo el set para almacenar los estados visitados
    explored = set()

    node_explored = 0

    while len(frontier) != 0:
        node = frontier.pop()
        node_explored += 1

        # Agregando los nodos explorados al set
        explored.add(node.state)

        # Chequeando si el objetivo fue alcanzado
        if problem.goal_test(node.state):
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": len(frontier),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost,  # Assuming this exists
            }
            return node, metrics

        for next_node in node.expand(problem):
            if next_node.state not in explored and next_node not in frontier:
                frontier.append(next_node)

    # Metricas para una busqueda no exitosa
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": len(frontier),
        "max_depth": node.depth if len(frontier) > 0 else 0,
        "cost_total": None,
    }
    return None, metrics


In [3]:
solution, metrics = depth_first_search(number_disks=5)

In [4]:
# Metricas vinculadas a la solucion encontrada por el algoritmo de busqueda

for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: True
nodes_explored: 82
states_visited: 82
nodes_in_frontier: 41
max_depth: 81
cost_total: 81.0


In [ ]:
# Secuencia de estados explorada por el algoritmo de busqueda hasta llegar a la solucion

for nodos in solution.path():
    print(nodos)

In [6]:
# Secuencia de moviemientos desde el estado inicial hasta el estado final

for act in solution.solution():
    print(act)

Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 4 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 3 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 5 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 3 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from

ENUNCIADO 6

In [7]:
%%timeit 
solution, metrics = depth_first_search(number_disks=5)

13.9 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
import tracemalloc

tracemalloc.start()

solution, metrics = depth_first_search(number_disks=5)

# Para medir memoria consumida usamos el pico de memoria
_, memory_peak = tracemalloc.get_traced_memory()
memory_peak /= 1024*1024
tracemalloc.stop()

print(f"Pico de memoria ocupada: {round(memory_peak, 2)} [MB]", )

Pico de memoria ocupada: 0.17 [MB]


ENUNCIADO 7

In [9]:
import numpy as np  # Para calcular promedio y desviación estándar

def compare_with_optimal(number_disks=5, runs=10):
    optimal_solution = 2**number_disks - 1  # Fórmula para solución óptima
    moves_used = []  # Lista para almacenar los movimientos usados

    for _ in range(runs):
        solution, metrics = depth_first_search(number_disks=number_disks)
        if metrics["solution_found"]:
            moves_used.append(metrics["cost_total"])  # Suponiendo que cost_total almacena movimientos totales
    
    # Calcular promedio y desviación estándar
    avg_moves = np.mean(moves_used)
    std_dev_moves = np.std(moves_used)
    difference_from_optimal = avg_moves - optimal_solution  # Diferencia respecto a la solución óptima

    results = {
        "optimal_solution": optimal_solution,
        "average_moves": avg_moves,
        "std_dev_moves": std_dev_moves,
        "difference_from_optimal": difference_from_optimal,
    }

    return results

# Ejecutar la comparación
results = compare_with_optimal(number_disks=5, runs=10)

# Imprimir los resultados
for metric, value in results.items():
    print(f"{metric}: {value}")


optimal_solution: 31
average_moves: 81.0
std_dev_moves: 0.0
difference_from_optimal: 50.0


In [ ]:
PRODUCCION DE ARCHIVOS JSON PARA EL SIMULADOR

In [10]:
solution.generate_solution_for_simulator()